In [ ]:
import gym
import json
import time
import pandas as pd
from datetime import timedelta
from utils import make_dict
import Publisher as pub

In [ ]:
def main():
    # Initialize the simulation environment
    env = gym.make('Lab-Building-default-v0')
    publisher = pub.MyPublisher('Lab-Building')
    publisher.start()

    # Define observation variables
    obs_name = [
        "T_ext", "Wind_Speed", "Wind_Direction", "Reception",
        "Bathroom1", "Bathroom2", "Bathroom3", "Corridor", "Office", 
        "Electricity", "DistrictHeating", "DistrictCooling", "Bath1_shade", 
        "Classroom1_shade1", "Classroom1_shade2", "Bath_shade", "Office_shade1", "Office_shade2", 
        "Classroom2_shade1", "Classroom2_shade2"
    ]
    
    tot_days = 14  # Total number of days/episodes
    n_step_hour = 4  # Timesteps in one hour
    n_step = n_step_hour * 24
    time_step, observations, is_terminal, _ = env.reset()
    start_time = pd.to_datetime(f"{env.start_year}-{env.start_mon}-{env.start_day}")
    cur_time = start_first = start_time
    obs_dict = make_dict(obs_name, observations)

    # Simulation loop
    for i_episode in range(tot_days):
        for t in range(n_step):
            input_actions = []  # Define actions to take (empty list for now)
            time_step, observations, is_terminal, _ = env.step(input_actions)
            cur_time = start_time + timedelta(seconds=time_step)

            # Save for record
            obs_dict = make_context(obs_name, observations)
            print(obs_dict)  # Printing observations for debugging

            # Send data
            send_data(publisher, obs_dict, cur_time)
            time.sleep(2)  # Sleep to simulate time delay

    env.end_env()

In [ ]:
def send_data(publisher, obs_dict, cur_time):
    """Helper function to send observation data."""
    for key, value in obs_dict.items():
        payload = {
            "location": "MyBuilding",
            "measurement": "test",
            "node": key,
            "time_stamp": str(cur_time),
            "value": str(value)
        }
        publisher.myPublish("ict4bd", json.dumps(payload))
        time.sleep(0.01)  # Small delay between sending data

In [ ]:
if __name__ == '__main__':
    main()